In [1]:
import CMZoom_utility as cmz
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import astrodendro
import os
from astropy import wcs
from astrodendro import Dendrogram, pp_catalog
from astrodendro.analysis import PPStatistic
from astropy.table import Table, hstack, Column
from astropy.utils.console import ProgressBar
import reproject
import pyregion
print(np.__version__)
import aplpy

1.16.5


In [2]:
path = os.path.expanduser('/Users/hph/cmzoom_catalog/catalog_parameter_study/')

In order to characterize the variation in the effective radius of each structure we performed a simple parameter study, modulating the initial dendrogram's minimum $\delta$ parameter. As described in section $N$, this parameter characterizes the minimum required difference in intensity between an independent structure and the local minima that separated that structure from any adjacent independent structure in the dendrogram. A separate dendrogram was constructed for three $\delta$ values: $\delta = 1$, our fidelity value (the one used the CMZoom catalog) of $\delta = 2$, and $\delta = 3$. Those dendrograms are then processed with the same pruning as the catalog to produce a low-$\delta$ catalog, our fidelity catalog, and a high-$\delta$ catalog. The effect that this variation has on the catalog results is described in figures $N$ and $N+1$.

Two cases arise in our catalog for variations in $\delta$: leaves can change shape, and therefore also integrated flux, or two nearby leaves can merge together or divide. The former case is more common than the latter, but both cases are fairly rare and only occur for lower flux sources. The effect that changes in $\delta$ have on the catalog as a whole are represented in figure $N+3$. 

In [3]:
########################################################################################
### define function for initial catalog table (identical to CMZoom_run_catalog.ipynb)
########################################################################################

def run_characterization(Target, DStructure, final_noisemap,pp,pm,filesuffix, path):
    InputMap = fits.open(Target+'.fits')[0]
    
    HeaderMod = InputMap.header.copy()

    PixelAreaArcsec = 3600. * abs(InputMap.header['CDELT1']) * 3600. * abs(InputMap.header['CDELT2'])
    Metadata = {}
    Metadata['data_unit'] = u.Jy / u.sr
    Metadata['spatial_scale'] =  PixelAreaArcsec**0.5 * u.arcsec

    Metadata['wcs'] = wcs.WCS(header=HeaderMod).celestial
    #Actually build the catalog from the initial dendrogram
    Catalogue = astrodendro.pp_catalog(DStructure, Metadata)
    #Customize catalog with units and coordinate system
    Catalogue['x_cen'].unit = u.deg
    Catalogue['y_cen'].unit = u.deg
    Catalogue.rename_column('_idx', 'index')
    Catalogue.rename_column('flux', 'flux_integrated')
    Catalogue.rename_column('x_cen', 'glon')
    Catalogue.rename_column('y_cen', 'glat')
    Catalogue['flux_integrated'].unit = u.Jy
    Catalogue['flux_integrated'] = Catalogue['flux_integrated']
    Catalogue['r_eff'] = (Catalogue['area_exact']/np.pi)**0.5
    Catalogue['r_eff'].unit = u.arcsec
    Catalogue['glon'].unit = u.deg
    Catalogue['glat'].unit = u.deg
    
    # add a 'noise' column to the catalog
    keys = ['testflux_integrated', 'flux_integrated_bgsub','min_flux_integrated', 'noise', 'is_leaf', 'peak_cont_flux', 'min_cont_flux', 'mean_cont_flux','npix']
    columns = {k:[] for k in (keys)}
    
    for ii, row in enumerate(ProgressBar(Catalogue)):
        structure = dend[row['index']]
        assert structure.idx == row['index'] == ii
        dend_inds = structure.indices()
        columns['noise'].append(final_noisemap[dend_inds].mean())
        columns['is_leaf'].append(structure.is_leaf)
        peakflux = data[dend_inds].max()
        testflux = np.nansum(data[dend_inds])
        testflux_integrated = testflux*JyperSr_to_JyperPix
        minflux = data[dend_inds].min()
        numpix = np.shape(dend_inds)[1]
        backgroundflux = minflux*numpix*JyperSr_to_JyperPix
        columns['flux_integrated_bgsub'].append(Catalogue['flux_integrated'][ii] - backgroundflux)
        columns['peak_cont_flux'].append(peakflux)
        columns['min_cont_flux'].append(data[dend_inds].min())
        columns['mean_cont_flux'].append(data[dend_inds].mean())
        columns['npix'].append(numpix)
        columns['min_flux_integrated'].append(backgroundflux)
        columns['testflux_integrated'].append(testflux_integrated)
        
    for k in columns:
        if k not in Catalogue.keys():
            Catalogue.add_column(Column(name=k, data=columns[k]))
    
    Catalogue['peak_cont_flux'].unit = u.Jy / u.sr
    Catalogue['mean_cont_flux'].unit = u.Jy / u.sr
    Catalogue['mean_cont_flux'].unit = u.Jy / u.sr
    
    #pruning
    print("Pruning...")
    cat_mask = (Catalogue['is_leaf'] &
                (Catalogue['peak_cont_flux']>pp*Catalogue['noise']) &
                (Catalogue['mean_cont_flux']>pm*Catalogue['noise']))
    pruned_ppcat = Catalogue[cat_mask]
    mask = dend.index_map.copy()
    for ii in ProgressBar(list(range(len(Catalogue)))):
        if ii not in pruned_ppcat['index']:
            mask[mask == ii] = -1
    print(mask.shape)
    print(mask)
    outf = fits.PrimaryHDU(data=mask, header=HeaderMod)
    print(outf)
    print("Writing mask file...")
    outf.writeto(path+'dendrogram_mask_pruned'+filesuffix+'.fits', overwrite=True)
    
    print("Writing region file...")
    with open(path+"raw_dendrogram_cores"+filesuffix+".reg", 'w') as fh:
        fh.write("galactic\n")
        for row in pruned_ppcat:
            fh.write("ellipse({glon}, {glat}, {major_sigma}\", "
                     "{minor_sigma}\", {position_angle}) # text={{{index}}}\n"
                     .format(**dict(zip(row.colnames, row))))
        
    print("Region file complete!")
    
    print("Writing data table...")        
    OutputPath = path + 'catalog'+'_datatab'+filesuffix+'.fits'
    if os.path.isfile(OutputPath):
        os.remove(OutputPath)
    pruned_ppcat.write(OutputPath,
                    format='fits')
    print("Data table written!")
    

In [4]:
### run two (additional) catalogs, save to a new folder
### use delta = 1, delta = 2, delta = 3
#ASSIGN PARAMETERS

rms = 3.*10**6
dend_min_v = 3
dend_min_d1 = 1
dend_min_d2 = 2
dend_min_d3 = 3
dend_min_pix = 17
prune_min_peak = 6
prune_min_mean = 2


date = '_2-11-20'
directory = '/Users/hph/cmzoom_catalog/catalog_parameter_study/'
target = directory+'CMZoom_continuum_pbcor.fits'
filesuffix1 = '_rms3e6_dv'+str(dend_min_v)+'_dd'+str(dend_min_d1)+'_dp'+str(dend_min_pix)+'_pp'+str(prune_min_peak)+'_pm'+str(prune_min_mean)+'_gal'+date
filesuffix2 = '_rms3e6_dv'+str(dend_min_v)+'_dd'+str(dend_min_d2)+'_dp'+str(dend_min_pix)+'_pp'+str(prune_min_peak)+'_pm'+str(prune_min_mean)+'_gal'+date
filesuffix3 = '_rms3e6_dv'+str(dend_min_v)+'_dd'+str(dend_min_d3)+'_dp'+str(dend_min_pix)+'_pp'+str(prune_min_peak)+'_pm'+str(prune_min_mean)+'_gal'+date


sma = fits.open(target)
data = sma[0].data
HeaderMod = sma[0].header

mywcs = wcs.WCS(HeaderMod)

print( "Running dendrogram...")
dend1 = astrodendro.Dendrogram.compute(data, min_value=float(dend_min_v)*rms, min_delta=float(dend_min_d1)*rms,
                                      min_npix=dend_min_pix,
                                      wcs=mywcs)
print( "Dendrogram 1 complete!")
#Save the dendrogram to the working directory (temporary)
dend1.save_to(directory+'/prototype_dendrogram'+filesuffix1+'.fits')

dend2 = astrodendro.Dendrogram.compute(data, min_value=float(dend_min_v)*rms, min_delta=float(dend_min_d2)*rms,
                                      min_npix=dend_min_pix,
                                      wcs=mywcs)
print( "Dendrogram 2 complete!")
#Save the dendrogram to the working directory (temporary)
dend2.save_to(directory+'/prototype_dendrogram'+filesuffix2+'.fits')

dend3 = astrodendro.Dendrogram.compute(data, min_value=float(dend_min_v)*rms, min_delta=float(dend_min_d3)*rms,
                                      min_npix=dend_min_pix,
                                      wcs=mywcs)
print( "Dendrogram 3 complete!")
#Save the dendrogram to the working directory (temporary)
dend3.save_to(directory+'/prototype_dendrogram'+filesuffix3+'.fits')

noisemap = fits.open(directory+'CMZoom_noisemap_JySr_k14_final.fits')
noisemap_data = noisemap[0].data
print( "Running characterization...")
#Perform pruning and save catalog to working directory (temporary)
run_characterization(target,dend1,noisemap_data,float(prune_min_peak),float(prune_min_mean),filesuffix1,directory)
run_characterization(target,dend2,noisemap_data,float(prune_min_peak),float(prune_min_mean),filesuffix2,directory)
run_characterization(target,dend3,noisemap_data,float(prune_min_peak),float(prune_min_mean),filesuffix3,directory)
print( "Characterization complete!")


Running dendrogram...


/Users/hph/anaconda/envs/cmzoom3/lib/python3.6/site-packages/astrodendro/dendrogram.py:177: RuntimeWarning: invalid value encountered in greater
  keep = self.data > min_value


Dendrogram 1 complete!


Dendrogram 2 complete!
Dendrogram 3 complete!
Running characterization...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/hph/cmzoom_catalog/catalog_parameter_study/CMZoom_continuum_pbcor.fits.fits'